In [ ]:
import pandas as pd
import numpy as np
# Display settings
pd.set_option('display.max_rows',40000) # Nb of rows to display
pd.set_option('display.max_columns',1000) # Nb of columns to display
pd.set_option('display.max_colwidth',1000) # Max number of characters in a cell before being truncated

In [ ]:
# Make sure your file is named like FEC_COMPANY_NAME_YEAR.FORMAT
COMPANY_NAME = 'METRON'
YEAR = '2021'
FORMAT = 'txt'

Run either one of the three following cells to import the data.          
To know if it worked, run the df.head() cell and check the result.      
It is possible that none of them works properly. You will have to find another solution by iterating.

In [ ]:
df = pd.read_csv('FEC_' + COMPANY_NAME + '_' + YEAR + '.' + FORMAT, 
                 delimiter='\t', encoding='utf-8',error_bad_lines=False, dtype=object)

In [ ]:
df = pd.read_csv('FEC_' + COMPANY_NAME + '_' + YEAR + '.' + FORMAT, 
                 delimiter='\t', encoding='latin-1', error_bad_lines=False)

In [ ]:
df = pd.read_csv('FEC_' + COMPANY_NAME + '_' + YEAR + '.' + FORMAT, 
                 delimiter=';', encoding='windows-1252', error_bad_lines=False)

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
# To use if you want to replace a character by another in the entire dataframe
# This can be useful in case of encoding issues
character_to_replace = "xx"
replace_by = "xx"
#df.replace(to_replace=character_to_replace, value=replace_by, regex=True, inplace=True)

df.columns should return something like this:

Index(['JournalCode', 'JournalLib', 'EcritureNum', 'EcritureDate', 'CompteNum',    
       'CompteLib', 'CompAuxNum', 'CompAuxLib', 'PieceRef', 'PieceDate',     
       'EcritureLib', 'Debit', 'Credit', 'EcritureLet', 'DateLet ',    
       'ValidDate', 'Montantdevise', 'Idevise'],    
      dtype='object')

In [ ]:
df.columns

In [ ]:
# To use if you need to rename some columns
#df.rename(columns={"JournalLib                    ": "JournalLib", 
#                   "CompteNum    ": "CompteNum", 
#                   "CompteLib                          ": "CompteLib",
#                   "CompAuxNum   ": "CompAuxNum",
#                   "CompAuxLib                         ": "CompAuxLib",
#                   "PieceRef    ": "PieceRef",
#                   "EcritureLib                        ": "EcritureLib",
#                   "        Debit": "Debit",
#                   "       Credit": "Credit"},
#                   inplace=True)

In [ ]:
# Oldest date
min(df["EcritureDate"])

In [ ]:
# Most recent date
max(df["EcritureDate"])

Make sure the dates span 12 months, and remember to update the "FEC period" property on the Notion card of the client with these dates.

In [ ]:
df["JournalLib"].value_counts()

In [ ]:
df.CompteNum.value_counts()

In [ ]:
df.CompteLib.value_counts()

In [ ]:
# Save the dataframe to excel (helps when there are encoding problems)
df.to_excel('FEC_' + COMPANY_NAME + '_' + YEAR + '.xlsx', index=False)

### Above is just general codes to turn txt file into excel, the part below will output a excel will debit 408&486, for first step of FNP&CCA processing

In [ ]:
import sqldf as sql

### Modify 'A nouveaux' if in your file it's called something else

In [ ]:
df = df.loc[df.JournalLib!='A nouveaux']

In [ ]:
query_debit_408 = """
        SELECT * from df
        WHERE CompteNum LIKE '408%' and Debit!='0,00';
"""

In [ ]:
Debit_Compte_408 = sql.run(query_debit_408)

In [ ]:
query_debit_486 = """
        SELECT * from df
        WHERE CompteNum LIKE '486%' and Debit!='0,00';
"""

In [ ]:
Debit_Compte_486 = sql.run(query_debit_486)

In [ ]:
def output(x,y):
    temp = """
                SELECT* FROM df
                WHERE EcritureNum=='{0}' and EcritureDate=='{1}'
           """.format(x,y)
    return sql.run(temp)

In [ ]:
df_list = []
for i in range(len(Debit_Compte_408)):
    x = Debit_Compte_408.loc[i,'EcritureNum']
    y = Debit_Compte_408.loc[i, 'EcritureDate']
    temp = output(x,y)
    df_list.append(temp)

In [ ]:
df_output = pd.concat(df_list)

In [ ]:
df_output = df_output.reset_index()

In [ ]:
df_output

In [ ]:
df_output.to_excel('FEC_' + COMPANY_NAME + '_FNP&CCA_1st_step_' + YEAR + '.xlsx', index=False)